In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/vimu/Documents/Data Science/SDS/Github/TimeSeriesAnalysis/Data/Energy_consumption.csv',index_col= "Timestamp",parse_dates=True)

In [3]:
df.head()

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption
Timestamp,,,,,,,,,,
2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,On,Off,2.774699,Monday,No,75.364373
2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,On,On,21.831384,Saturday,No,83.401855
2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,Off,Off,6.764672,Sunday,No,78.270888
2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,Off,On,8.623447,Wednesday,No,56.519850
2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,On,Off,3.071969,Friday,No,70.811732


In [4]:
df['DayOfWeek'] = df.index.day_name()

In [5]:
df.head()

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption
Timestamp,,,,,,,,,,
2022-01-01 00:00:00,25.139433,43.431581,1565.693999,5,On,Off,2.774699,Saturday,No,75.364373
2022-01-01 01:00:00,27.731651,54.225919,1411.064918,1,On,On,21.831384,Saturday,No,83.401855
2022-01-01 02:00:00,28.704277,58.907658,1755.715009,2,Off,Off,6.764672,Saturday,No,78.270888
2022-01-01 03:00:00,20.080469,50.371637,1452.316318,1,Off,On,8.623447,Saturday,No,56.519850
2022-01-01 04:00:00,23.097359,51.401421,1094.130359,9,On,Off,3.071969,Saturday,No,70.811732


In [10]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [11]:
def create_lag_features(data, target_col, n_lags):
    df = data.copy()
    for lag in range(1, n_lags + 1):
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)
    df.dropna(inplace=True)
    return df

In [12]:
n_lags = 10  # You can try 5, 10, etc.
df_lagged = create_lag_features(df, 'EnergyConsumption', n_lags)

In [13]:
df_lagged

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption,EnergyConsumption_lag_1,EnergyConsumption_lag_2,EnergyConsumption_lag_3,EnergyConsumption_lag_4,EnergyConsumption_lag_5,EnergyConsumption_lag_6,EnergyConsumption_lag_7,EnergyConsumption_lag_8,EnergyConsumption_lag_9,EnergyConsumption_lag_10
Timestamp,,,,,,,,,,,,,,,,,,,,
2022-01-01 10:00:00,28.064814,36.451472,1341.467129,2,Off,Off,0.352238,Saturday,Yes,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888,83.401855,75.364373
2022-01-01 11:00:00,23.422546,30.527342,1604.418355,6,On,On,19.529548,Saturday,Yes,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888,83.401855
2022-01-01 12:00:00,25.388888,47.601018,1244.618914,1,On,Off,21.797444,Saturday,Yes,84.144776,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888
2022-01-01 13:00:00,20.058738,41.861642,1806.052632,2,Off,Off,6.384949,Saturday,Yes,60.022519,84.144776,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850
2022-01-01 14:00:00,26.731525,37.297870,1419.749014,6,Off,Off,12.074223,Saturday,Yes,81.183188,60.022519,84.144776,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-11 11:00:00,28.619382,48.850160,1080.087000,5,Off,Off,21.194696,Friday,No,82.306692,89.964064,83.299462,93.639644,64.903078,74.638956,71.775781,86.780203,73.621237,62.729873,85.271028
2022-02-11 12:00:00,23.836647,47.256435,1705.235156,4,Off,On,25.748176,Friday,Yes,66.577320,82.306692,89.964064,83.299462,93.639644,64.903078,74.638956,71.775781,86.780203,73.621237,62.729873
2022-02-11 13:00:00,23.005340,48.720501,1320.285281,6,Off,On,0.297079,Friday,Yes,72.753471,66.577320,82.306692,89.964064,83.299462,93.639644,64.903078,74.638956,71.775781,86.780203,73.621237


In [14]:
X = df_lagged.drop(['EnergyConsumption'], axis=1)
y = df_lagged['EnergyConsumption']

# 80-20 time-based split
split = int(len(df_lagged) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

In [17]:
X_train

,Temperature,Humidity,SquareFootage,Occupancy,HVACUsage,LightingUsage,RenewableEnergy,DayOfWeek,Holiday,EnergyConsumption_lag_1,EnergyConsumption_lag_2,EnergyConsumption_lag_3,EnergyConsumption_lag_4,EnergyConsumption_lag_5,EnergyConsumption_lag_6,EnergyConsumption_lag_7,EnergyConsumption_lag_8,EnergyConsumption_lag_9,EnergyConsumption_lag_10
Timestamp,,,,,,,,,,,,,,,,,,,
2022-01-01 10:00:00,28.064814,36.451472,1341.467129,2,Off,Off,0.352238,Saturday,Yes,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888,83.401855,75.364373
2022-01-01 11:00:00,23.422546,30.527342,1604.418355,6,On,On,19.529548,Saturday,Yes,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888,83.401855
2022-01-01 12:00:00,25.388888,47.601018,1244.618914,1,On,Off,21.797444,Saturday,Yes,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850,78.270888
2022-01-01 13:00:00,20.058738,41.861642,1806.052632,2,Off,Off,6.384949,Saturday,Yes,84.144776,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732,56.519850
2022-01-01 14:00:00,26.731525,37.297870,1419.749014,6,Off,Off,12.074223,Saturday,Yes,60.022519,84.144776,73.278670,82.274434,77.992214,78.206236,74.131906,76.165791,84.321885,70.811732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-03 05:00:00,22.762376,43.312725,1803.359257,6,On,Off,7.930077,Thursday,No,88.596723,69.193144,74.449984,88.546906,87.721537,69.565454,71.400421,81.743052,68.973114,70.069460
2022-02-03 06:00:00,28.533588,37.003065,1697.399120,1,Off,On,7.059707,Thursday,No,73.587276,88.596723,69.193144,74.449984,88.546906,87.721537,69.565454,71.400421,81.743052,68.973114
2022-02-03 07:00:00,23.939926,32.804434,1712.388364,5,Off,On,14.248994,Thursday,No,75.748232,73.587276,88.596723,69.193144,74.449984,88.546906,87.721537,69.565454,71.400421,81.743052
